<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Financial Packages

&copy; Dr. Yves J. Hilpisch | The Python Quants GmbH

http://tpq.io | [training@tpq.io](mailto:trainin@tpq.io) | [@dyjh](http://twitter.com/dyjh)

### Please use the "Python 3.10, Numpy 1.26.4 Pandas TA" kernel.

## `vectorbt` package

In [ ]:
!git clone https://github.com/tpq-classes/financial_packages.git
import sys
sys.path.append('financial_packages')


In [ ]:
import numpy as np
import pandas as pd
from pylab import plt
# plt.style.use('seaborn-v0_8')
plt.style.use('seaborn-v0_8')
%config InlineBackend.figure_format = 'svg'

In [ ]:
import warnings
warnings.simplefilter('ignore')

## Import

You can install the package via (see https://vectorbt.dev/):

    pip install vectorbt

You should install `plotly` as well and configure Jupyter Lab for the usage of it.

    pip install --upgrade plotly
    conda install nodejs
    jupyter labextension install jupyterlab-plotly
    jupyter labextension install plotlywidget

In [ ]:
import vectorbt as vbt

## Data

In [ ]:
url = 'https://certificate.tpq.io/findata.csv'

In [ ]:
raw = pd.read_csv(url, index_col=0, parse_dates=True)

In [ ]:
raw.info()

In [ ]:
#ticker = 'MSFT.O'
#ticker = 'AAPL.O'
ticker = 'GLD'
#ticker = '.SPX'

In [ ]:
data = pd.DataFrame(raw[ticker]).dropna()

In [ ]:
data.info()

In [ ]:
data.plot();

## Long Investment

In [ ]:
pf = vbt.Portfolio.from_holding(data)  # long investment

In [ ]:
pf.total_return()

In [ ]:
pf = vbt.Portfolio.from_holding(data, init_cash=15000)  # long investment

In [ ]:
pf.total_profit()

## SMA-Based Strategy 

### Long Only

In [ ]:
sma1 = 42
sma2 = 252

In [ ]:
SMA1 = vbt.MA.run(data, sma1, short_name='fast')

In [ ]:
# SMA1

In [ ]:
SMA2 = vbt.MA.run(data, sma2, short_name='slow')

In [ ]:
entries = SMA1.ma_crossed_above(SMA2)

In [ ]:
entries.value_counts()

In [ ]:
exits = SMA1.ma_crossed_below(SMA2)

In [ ]:
exits.value_counts()

In [ ]:
# vbt.Portfolio.from_signals?

In [ ]:
pf = vbt.Portfolio.from_signals(data, entries, exits)

In [ ]:
pf.total_return()

In [ ]:
pf.asset_returns().value_counts()

In [ ]:
pf.drawdowns.active_drawdown()

In [ ]:
pf.drawdowns.active_duration()

In [ ]:
pf.drawdowns.avg_drawdown()

In [ ]:
pf.drawdowns.avg_duration()

In [ ]:
pf.stats()

In [ ]:
pf.wrapper.columns

In [ ]:
pf[(sma1, sma2, ticker)].plot().show()

## Short Only

In [ ]:
short_entries = SMA1.ma_crossed_below(SMA2)

In [ ]:
short_exits = SMA1.ma_crossed_above(SMA2)

In [ ]:
pf = vbt.Portfolio.from_signals(data, short_entries=short_entries,
                                short_exits=short_exits)

In [ ]:
pf.total_return()

In [ ]:
pf.stats()

In [ ]:
pf[(sma1, sma2, ticker)].plot().show()

### Long-Short

In [ ]:
entries = SMA1.ma_crossed_above(SMA2)

In [ ]:
short_entries = SMA1.ma_crossed_below(SMA2)

In [ ]:
pf = vbt.Portfolio.from_signals(data, entries=entries,
                                short_entries=short_entries)

In [ ]:
pf.total_return()

In [ ]:
pf.stats()

In [ ]:
pf[(sma1, sma2, ticker)].plot().show()

## Random Signals

In [ ]:
# pf = vbt.Portfolio.from_random_signals?

### `n` Entries & Exits

In [ ]:
n = 15

In [ ]:
pf = vbt.Portfolio.from_random_signals(data, n=n)

In [ ]:
pf.total_benchmark_return()

In [ ]:
pf.total_return()

In [ ]:
pf.orders.count()

In [ ]:
pf.wrapper.columns

In [ ]:
pf[(n, ticker)].plot().show()

### Entry & Exit Probabilities

In [ ]:
p = 0.05

In [ ]:
pf = vbt.Portfolio.from_random_signals(data, entry_prob=p,
                                       exit_prob=p, param_product=True)

In [ ]:
pf.total_return()

In [ ]:
pf.orders.count()

In [ ]:
p * len(data)

In [ ]:
pf.wrapper.columns

In [ ]:
pf[(p, p, ticker)].plot().show()

## From "Any" Signal

1 = long signal | -1 = short signal

In [ ]:
signals = np.array(sum(15 * [150 * [1] + 100 * [-1]], []))[:len(data)]
len(signals)

In [ ]:
signals

In [ ]:
entries = np.where(signals == 1, True, False)
short_entries = np.where(signals == -1, True, False)

In [ ]:
entries = pd.DataFrame(entries, index=data.index)
short_entries = pd.DataFrame(short_entries, index=data.index)

In [ ]:
# entries

In [ ]:
# short_entries

In [ ]:
pf = vbt.Portfolio.from_signals(data, entries=entries,
                                short_entries=short_entries)

In [ ]:
pf.total_return()

In [ ]:
pf.total_benchmark_return()

In [ ]:
pf.wrapper.columns

In [ ]:
pf[ticker].plot().show()

## Multiple Strategies

### Multiple Strategy Configurations

In [ ]:
SMA1 = vbt.MA.run(data, [20, 20, 20], short_name='fast')

In [ ]:
SMA2 = vbt.MA.run(data, [40, 50, 60], short_name='slow')

In [ ]:
entries = SMA1.ma_crossed_above(SMA2)

In [ ]:
short_entries = SMA1.ma_crossed_below(SMA2)

In [ ]:
pf = vbt.Portfolio.from_signals(data, entries=entries,
                                short_entries=short_entries)

In [ ]:
pf.total_return()

In [ ]:
pf.drawdowns.max_drawdown()

In [ ]:
pf.drawdowns.max_duration()

In [ ]:
pf[(20, 60)].stats()

In [ ]:
pf[(20, 60)].plot().show()

### Multiple Instruments

In [ ]:
data = raw[[ticker, 'EUR=']].dropna()

In [ ]:
SMA1 = vbt.MA.run(data, [20, 20, 20], short_name='fast')

In [ ]:
SMA2 = vbt.MA.run(data, [40, 50, 60], short_name='slow')

In [ ]:
entries = SMA1.ma_crossed_above(SMA2)

In [ ]:
short_entries = SMA1.ma_crossed_below(SMA2)

In [ ]:
pf = vbt.Portfolio.from_signals(data, entries=entries,
                                short_entries=short_entries)

In [ ]:
pf.total_return()

In [ ]:
# pf_ = vbt.Portfolio.from_holding(data)  # long investment

In [ ]:
# pf_.total_return()

In [ ]:
pf.total_benchmark_return()

In [ ]:
pf.total_return()

In [ ]:
pf[(20, 50, 'EUR=')].stats()

In [ ]:
pf[(20, 50, 'EUR=')].plot().show()

### Multiple Periods

In [ ]:
mult_data, _ = data.vbt.range_split(n=3)

In [ ]:
mult_data.head()

In [ ]:
SMA1 = vbt.MA.run(mult_data, 10, short_name='fast')

In [ ]:
SMA2 = vbt.MA.run(mult_data, 20, short_name='slow')

In [ ]:
entries = SMA1.ma_crossed_above(SMA2)

In [ ]:
exits = SMA1.ma_crossed_below(SMA2)

In [ ]:
pf = vbt.Portfolio.from_signals(mult_data, entries, exits, init_cash=10000)

In [ ]:
pf.total_profit()

## Integration with `pandas-ta`

In [ ]:
data = raw['AMZN.O'].dropna()

In [ ]:
vbt.pandas_ta('SMA').run(data, [40, 60]).sma.tail()

## Integration with `quantstats`

In [ ]:
import quantstats as qs

In [ ]:
returns = data.vbt.to_returns()
returns.vbt.returns.qs.plot_snapshot()

In [ ]:
pf = vbt.Portfolio.from_holding(data)

In [ ]:
o = pf.returns().vbt.returns.qs

In [ ]:
o.cagr(periods=256)

In [ ]:
o.plot_monthly_heatmap()

In [ ]:
o.plot_snapshot()

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="mailto:training@tpq.io">training@tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> 